In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from qiskit.tools.jupyter import *
from qiskit import IBMQ
from qiskit import assemble
from qiskit.tools.monitor import job_monitor

from qiskit import pulse
from qiskit.pulse import Play
from qiskit.pulse import pulse_lib

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

In [2]:
backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

dt = backend_config.dt
print(f"Sampling time: {dt*1e9} ns")    # The configuration returns dt in seconds, so multiply by
                                        # 1e9 to get nanoseconds
    
backend_defaults = backend.defaults()

Sampling time: 0.2222222222222222 ns


In [3]:
# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

qubit = 0

center_frequency_Hz = backend_defaults.qubit_freq_est[qubit]        

print(f"Qubit {qubit} has an estimated frequency of {center_frequency_Hz / GHz} GHz.")

scale_factor = 1e-14

def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

freq_min = None
freq_max = None
drive_sigma_us = None
drive_samples_us = None
drive_sigma = None
drive_samples = None
drive_amp = None
drive_pulse = None
job = None
pi_amp = None
rough_qubit_frequency = 4.9745
precise_qubit_freq = 4.9745

Qubit 0 has an estimated frequency of 4.974437236363093 GHz.


In [4]:
num_rabi_points = 50

drive_amp_min = 0
drive_amp_max = 0.75
pi_pulse = None
job2 = None

def baseline_remove(values):
    return np.array(values) - np.mean(values)

In [26]:
def rabi_exp(drive_min, drive_max, num_points, rough_qubit_freq):
    global num_rabi_points                          # allows changes in variables to be reflected in other experiments
    global drive_amp_min
    global drive_amp_max
    global pi_pulse
    global job2                                     # job2.function() for rabi when retrieving data after function ends (jobid, experiment errors, etc.)
    global rough_qubit_frequency
    global pi_amp
    
    rough_qubit_frequency = rough_qubit_freq * GHz
    
    num_rabi_points = int(num_points)
    drive_amp_min = drive_min
    drive_amp_max = drive_max
    
    drive_amps = np.linspace(drive_amp_min, drive_amp_max, num_rabi_points)

    drive_chan = pulse.DriveChannel(qubit)
    meas_chan = pulse.MeasureChannel(qubit)
    acq_chan = pulse.AcquireChannel(qubit)
    
    meas_map_idx = None
    for i, measure_group in enumerate(backend_config.meas_map):
        if qubit in measure_group:
            meas_map_idx = i
            break
    assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
    
    inst_sched_map = backend_defaults.instruction_schedule_map
    measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])
    
    rabi_schedules = []
    for drive_amp in drive_amps:
        rabi_pulse = pulse_lib.gaussian(duration=drive_samples, amp=drive_amp, 
                                    sigma=drive_sigma, name=f"Rabi drive amplitude = {drive_amp}")
        this_schedule = pulse.Schedule(name=f"Rabi drive amplitude = {drive_amp}")
        this_schedule += Play(rabi_pulse, drive_chan)
        
        this_schedule += measure << this_schedule.duration
        rabi_schedules.append(this_schedule)
    
    rabi_schedules[-1].draw(label=True, scaling=1.0)
    
    num_shots_per_point = 1024

    rabi_experiment_program = assemble(rabi_schedules,
                                   backend=backend,
                                   meas_level=1,
                                   meas_return='avg',
                                   shots=num_shots_per_point,
                                   schedule_los=[{drive_chan: rough_qubit_frequency}]
                                                * num_rabi_points)

    job2 = backend.run(rabi_experiment_program)
    job_monitor(job2)
    
    rabi_results = job2.result(timeout=120)
    
    rabi_values = []
    for i in range(num_rabi_points):
        rabi_values.append(rabi_results.get_memory(i)[qubit]*scale_factor)

    rabi_values = np.real(baseline_remove(rabi_values))

    plt.xlabel("Drive amp [a.u.]")
    plt.ylabel("Measured signal [a.u.]")
    plt.scatter(drive_amps, rabi_values, color='black')
    plt.show()
    
    fit_params, y_fit = fit_function(drive_amps,
                                 rabi_values, 
                                 lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),
                                 [3, 0.1, 0.5, 0])

    plt.scatter(drive_amps, rabi_values, color='black')
    plt.plot(drive_amps, y_fit, color='red')

    drive_period = fit_params[2]

    plt.axvline(drive_period/2, color='red', linestyle='--')
    plt.axvline(drive_period, color='red', linestyle='--')
    plt.annotate("", xy=(drive_period, 0), xytext=(drive_period/2,0), arrowprops=dict(arrowstyle="<->", color='red'))
    plt.annotate("$\pi$", xy=(drive_period/2-0.03, 0.1), color='red')

    plt.xlabel("Drive amp [a.u.]", fontsize=15)
    plt.ylabel("Measured signal [a.u.]", fontsize=15)
    plt.show()
    
    pi_amp = abs(drive_period / 2)
    print(f"Pi Amplitude = {pi_amp}")
    
    pi_pulse = pulse_lib.gaussian(duration=drive_samples,
                              amp=pi_amp, 
                              sigma=drive_sigma,
                              name='pi_pulse')

In [6]:
def spectr_exp(frequency_min, frequency_max, sigma, amp):
    global freq_min                                # allows changes to variables to be reflected in other experiments
    global freq_max
    global drive_sigma_us
    global drive_samples_us
    global drive_sigma
    global drive_samples
    global drive_amp
    global drive_pulse
    global job1                                    # job1.function() for spectroscopy when retrieving data after function ends (jobid, experiment errors, etc.)
    global rough_qubit_frequency

    frequency_span_Hz = (frequency_max - frequency_min) * MHz
    # in steps of 1 MHz.
    frequency_step_Hz = 0.5 * MHz

    frequency_min = frequency_min * GHz
    frequency_max = frequency_max * GHz
    frequencies_GHz = np.arange(frequency_min / GHz, 
                            frequency_max / GHz, 
                            frequency_step_Hz / GHz)
    
    print(f"The sweep will go from {frequency_min / GHz} GHz to {frequency_max / GHz} GHz \
    in steps of {frequency_step_Hz / MHz} MHz.")
    
    # Drive pulse parameters (us = microseconds)
    drive_sigma_us = sigma                    
    drive_samples_us = sigma * 8

    drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us /dt)
    drive_samples = get_closest_multiple_of_16(drive_samples_us * us /dt)
    drive_amp = amp
    # Drive pulse samples
    drive_pulse = pulse_lib.gaussian(duration=drive_samples,
                                     sigma=drive_sigma,
                                     amp=drive_amp,
                                     name='freq_sweep_excitation_pulse')

    meas_map_idx = None
    for i, measure_group in enumerate(backend_config.meas_map):
        if qubit in measure_group:
            meas_map_idx = i
            break
    assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
    
    inst_sched_map = backend_defaults.instruction_schedule_map
    measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])
    
    drive_chan = pulse.DriveChannel(qubit)
    meas_chan = pulse.MeasureChannel(qubit)
    acq_chan = pulse.AcquireChannel(qubit)
    
    schedule = pulse.Schedule(name='Frequency sweep')
    schedule += Play(drive_pulse, drive_chan)
    schedule += measure << schedule.duration

    frequencies_Hz = frequencies_GHz*GHz
    schedule_frequencies = [{drive_chan: freq} for freq in frequencies_Hz]
    schedule.draw(label=True, scaling=0.8)                                            #doesn't draw schedules
    
    num_shots_per_frequency = 1024
    frequency_sweep_program = assemble(schedule,
                                   backend=backend, 
                                   meas_level=1,
                                   meas_return='avg',
                                   shots=num_shots_per_frequency,
                                   schedule_los=schedule_frequencies)
    
    job1 = backend.run(frequency_sweep_program)
    job_monitor(job1)
    frequency_sweep_results = job1.result(timeout=120)
    
    

    sweep_values = []
    for i in range(len(frequency_sweep_results.results)):
        res = frequency_sweep_results.get_memory(i)*scale_factor
        sweep_values.append(res[qubit])

    plt.scatter(frequencies_GHz, np.real(sweep_values), color='black')
    plt.xlim([min(frequencies_GHz), max(frequencies_GHz)])
    plt.xlabel("Frequency [GHz]")
    plt.ylabel("Measured signal [a.u.]")
    plt.show()
    
    fit_params, y_fit = fit_function(frequencies_GHz,
                                 np.real(sweep_values), 
                                 lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                 [5, 4.975, 1, 3]                        # initial parameters for curve_fit
                                )                                        # absolutely horrible for any other pulse parameters than the recommended, sometimes bad for that too
    
    plt.scatter(frequencies_GHz, np.real(sweep_values), color='black')
    plt.plot(frequencies_GHz, y_fit, color='red')
    plt.xlim([min(frequencies_GHz), max(frequencies_GHz)])

    plt.xlabel("Frequency [GHz]")
    plt.ylabel("Measured Signal [a.u.]")
    plt.show()
    
    A, rough_qubit_frequency, B, C = fit_params
    rough_qubit_frequency = rough_qubit_frequency*GHz
    print(f"Based on the parameters input, the new qubit frequency is " + str(round(rough_qubit_frequency/GHz, 5)) +" GHz.")

In [32]:
def ramsey_exp(pi_amp_set, timemax_set, timestep_set):
    global pi_amp               # allows changes to pi pulse amplitude to be applied to other experiments
    global timemax
    global timestep
    global precise_qubit_freq
    
    pi_amp = pi_amp_set
    timemax = timemax_set
    timestep = timestep_set
    
    time_max_us = timemax
    time_step_us = timestep
    times_us = np.arange(0.1, time_max_us, time_step_us)
    delay_times_dt = times_us * us / dt

    drive_amp = pi_amp_set / 2
    x90_pulse = pulse_lib.gaussian(duration=drive_samples,
                               amp=drive_amp, 
                               sigma=drive_sigma,
                               name='x90_pulse')
    
    drive_chan = pulse.DriveChannel(qubit)
    meas_chan = pulse.MeasureChannel(qubit)
    acq_chan = pulse.AcquireChannel(qubit)
    
    meas_map_idx = None
    for i, measure_group in enumerate(backend_config.meas_map):
        if qubit in measure_group:
            meas_map_idx = i
            break
    assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
    
    inst_sched_map = backend_defaults.instruction_schedule_map
    measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])
    
    ramsey_schedules = []

    for delay in delay_times_dt:
        this_schedule = pulse.Schedule(name=f"Ramsey delay = {delay * dt / us} us")
        this_schedule |= Play(x90_pulse, drive_chan)
        this_schedule |= Play(x90_pulse, drive_chan) << int(this_schedule.duration + delay)
        this_schedule |= measure << int(this_schedule.duration)

        ramsey_schedules.append(this_schedule)
        
    ramsey_schedules[0].draw(label=True, scaling=1.0)
    
    # Execution settings
    num_shots = 256

    detuning_MHz = 2 
    ramsey_frequency = round(rough_qubit_frequency + detuning_MHz * MHz, 6)
    ramsey_program = assemble(ramsey_schedules,
                             backend=backend,
                             meas_level=1,
                             meas_return='avg',
                             shots=num_shots,
                             schedule_los=[{drive_chan: ramsey_frequency}]*len(ramsey_schedules)
                            )
    
    job = backend.run(ramsey_program)
    job_monitor(job)
    
    ramsey_results = job.result(timeout=120)
    
    ramsey_values = []
    for i in range(len(times_us)):
        ramsey_values.append(ramsey_results.get_memory(i)[qubit]*scale_factor)
    
    plt.scatter(times_us, np.real(ramsey_values), color='black')
    plt.xlim(0, np.max(times_us))
    plt.title("Ramsey Experiment", fontsize=15)
    plt.xlabel('Delay between X90 pulses [$\mu$s]', fontsize=15)
    plt.ylabel('Measured Signal [a.u.]', fontsize=15)
    plt.show()
    
    fit_params, y_fit = fit_function(times_us, np.real(ramsey_values),
                                 lambda x, A, del_f_MHz, C, B: (
                                          A * np.cos(2*np.pi*del_f_MHz*x - C) + B
                                         ),
                                 [5, 1./0.4, 0, 0.25]
                                )

    _, del_f_MHz, _, _, = fit_params

    plt.scatter(times_us, np.real(ramsey_values), color='black')
    plt.plot(times_us, y_fit, color='red', label=f"df = {del_f_MHz:.2f} MHz")
    plt.xlim(0, np.max(times_us))
    plt.xlabel('Delay between X90 pulses [$\mu$s]', fontsize=15)
    plt.ylabel('Measured Signal [a.u.]', fontsize=15)
    plt.title('Ramsey Experiment', fontsize=15)
    plt.legend()
    plt.show()
    
    precise_qubit_freq = rough_qubit_frequency + (del_f_MHz - detuning_MHz) * MHz
    print(f"Our updated qubit frequency is now {round(precise_qubit_freq/GHz, 6)} GHz. "
      f"It used to be {round(rough_qubit_frequency / GHz, 6)} GHz")

In [14]:
def t1_exp(timemax_set, timestep_set):
    time_max_us = timemax_set                #different names used to avoid confusion with previous experiment globals
    time_step_us = timestep_set
    times_us = np.arange(1, time_max_us, time_step_us)
    delay_times_dt = times_us * us / dt
    
    drive_chan = pulse.DriveChannel(qubit)
    meas_chan = pulse.MeasureChannel(qubit)
    acq_chan = pulse.AcquireChannel(qubit)
    
    meas_map_idx = None                      # haven't tried if measure set can be done outside of functions, needed for all experiments
    for i, measure_group in enumerate(backend_config.meas_map):
        if qubit in measure_group:
            meas_map_idx = i
            break
    assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
    
    inst_sched_map = backend_defaults.instruction_schedule_map
    measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])
    
    t1_schedules = []
    for delay in delay_times_dt:
        this_schedule = pulse.Schedule(name=f"T1 delay = {delay * dt/us} us")
        this_schedule += Play(pi_pulse, drive_chan)
        this_schedule |= measure << int(delay)
        t1_schedules.append(this_schedule)
        
    sched_idx = 0
    t1_schedules[sched_idx].draw(label=True, scaling=1.0)
    
    num_shots = 256

    t1_experiment = assemble(t1_schedules,
                         backend=backend, 
                         meas_level=1,
                         meas_return='avg',
                         shots=num_shots,
                         schedule_los=[{drive_chan: rough_qubit_frequency}] * len(t1_schedules))
    
    job = backend.run(t1_experiment)
    job_monitor(job)
    
    t1_results = job.result(timeout=120)
    
    t1_values = []
    for i in range(len(times_us)):
        t1_values.append(t1_results.get_memory(i)[qubit]*scale_factor)
    t1_values = np.real(t1_values)

    plt.scatter(times_us, t1_values, color='black') 
    plt.title("$T_1$ Experiment", fontsize=15)
    plt.xlabel('Delay before measurement [$\mu$s]', fontsize=15)
    plt.ylabel('Signal [a.u.]', fontsize=15)
    plt.show()
    
    # Fit the data
    fit_params, y_fit = fit_function(times_us, t1_values, 
            lambda x, A, C, T1: (A * np.exp(-x / T1) + C),
            [-3, 3, 100]
            )

    _, _, T1 = fit_params

    plt.scatter(times_us, t1_values, color='black')
    plt.plot(times_us, y_fit, color='red', label=f"T1 = {T1:.2f} us")
    plt.xlim(0, np.max(times_us))
    plt.title("$T_1$ Experiment", fontsize=15)
    plt.xlabel('Delay before measurement [$\mu$s]', fontsize=15)
    plt.ylabel('Signal [a.u.]', fontsize=15)
    plt.legend()
    plt.show()

In [17]:
# spectroscopy experiment widget
print("Spectroscopy Experiment:")
interact_manual(spectr_exp,   #all of the default values for the widgets are the textbook suggested values
    frequency_min = widgets.FloatSlider(
    value=4.970,                    # default slider value
    min=4.955,
    max=4.995,
    step=0.001,
    description='Min Freq:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',       # can make all sliders vertical, slightly more annoying to use
    readout=True,                   # checks current slider value
    readout_format='.3f',           # number of digits after decimal displayed
    ),
    frequency_max = widgets.FloatSlider(
    value=4.980,
    min=4.955,
    max=4.995,
    step=0.001,
    description='Max Freq:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    ),
    sigma = widgets.FloatSlider(
    value=0.075,
    min=0.06,
    max=0.09,
    step=0.005,
    description='Sigma:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    ),
    amp = widgets.FloatSlider(
    value=0.3,
    min=0.1,
    max=0.5,
    step=0.05,
    description='Amp:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    ))

# rabi experiment widget
print("Rabi Experiment:")
interact_manual(rabi_exp,
    drive_min = widgets.FloatSlider(
    value=0,
    min=0,
    max=0.75,
    step=0.05,
    description='Min Amp:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    ),
    drive_max = widgets.FloatSlider(
    value=0.75,
    min=-0,
    max=1,
    step=0.05,
    description='Max Amp:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    ),
    num_points = widgets.FloatSlider(
    value=50,
    min=10,
    max=100,
    step=5,
    description='Points:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',   # glitch with IntSlider widget, used float with 0 places after decimal
    ),
    rough_qubit_freq = widgets.BoundedFloatText(
    value=rough_qubit_frequency/GHz,
    min=4.955,
    max=4.995,
    step=0.00001,
    description='Frequency',
    disabled=False,
))

# ramsey experiment widget
print("Ramsey Experiment:")
interact_manual(ramsey_exp,
    pi_amp_set = widgets.BoundedFloatText(
    value=pi_amp,
    min=0,
    max=1,
    step=0.000000000000001,
    description='Pi Amp:',
    disabled = False
    ),
    timemax_set = widgets.FloatSlider(
    value=1.8,
    min=1,
    max=2,
    step=0.1,
    description='Time Max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    ),
    timestep_set = widgets.FloatSlider(
    value=0.025,
    min=0,
    max=0.05,
    step=0.005,
    description='Time Step:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
))

# t1 widget
print("T1 Measurement:")
interact_manual(t1_exp,
    timemax_set = widgets.FloatSlider(
    value=450,
    min=300,
    max=500,
    step=25,
    description='Time Max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
    ),
    timestep_set = widgets.FloatSlider(
    value=6,
    min=1,
    max=10,
    step=1,
    description='Time Step:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
))

Spectroscopy Experiment:


interactive(children=(FloatSlider(value=4.97, continuous_update=False, description='Min Freq:', max=4.995, min…

Rabi Experiment:


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Min Amp:', max=0.75, step=0…

Ramsey Experiment:


interactive(children=(BoundedFloatText(value=0.3, description='Pi Amp:', max=1.0, step=1e-15), FloatSlider(val…

T1 Measurement:


interactive(children=(FloatSlider(value=450.0, continuous_update=False, description='Time Max:', max=500.0, mi…

<function __main__.t1_exp(timemax_set, timestep_set)>

In [ ]:
# future implementations:
# schedule drawing - doesn't show in widget output despite being included in function
# randomized benchmarking
# better curve fitting
# competitive scoring
# possible integration with textbook explanations of experiments?